In [ ]:
%reload_ext autoreload
%autoreload 2

### Scores for bias Gen

In [ ]:
import evaluation

metrics_file_path = './Final_Test_MetricsBase_CNN.npz'

evaluation.read_socres(metrics_file_path, conf_m_title='Concusion Matrix for Base_CNN')
evaluation.read_socres_gen(metrics_file_path, conf_m_title='Gen Concusion Matrix for Base_CNN')
evaluation.read_socres_race(metrics_file_path, conf_m_title='Race Concusion Matrix for Base_CNN')

In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as T
import matplotlib.pyplot as plt

from data_process.DatasetHelper import label_map
from data_process.DatasetHelper import label_map_new
from data_process.DatasetHelper import ImageDataset

data = ImageDataset('./data/aug_3')
data_loader = torch.utils.data.DataLoader(data, batch_size=100000, shuffle=True, num_workers=1)

for data, labels in data_loader:
    data_X, data_y = data.float(), labels
    
data_y = data_y.numpy()

label: 0 with 1000 images, named as "cloth_mask"
label: 1 with 1000 images, named as "no_face_mask"
label: 2 with 1000 images, named as "surgical_mask"
label: 3 with 1000 images, named as "n95_mask"
label: 4 with 1000 images, named as "mask_worn_incorrectly"


In [2]:
from evaluation import downgrade_argmax_target_to_gen, mask_label_name_list
from data_process.DatasetHelper import label_map_new_gen

gen_label_list = [downgrade_argmax_target_to_gen(y) for y in data_y]

d = {}

for l in gen_label_list:
    l_name = label_map_new_gen[l]
    if d.get(l_name) == None:
        d[l_name] = 0
    d[l_name] += 1

print(d)

gen_arr = {'m': [d[mask_male_label] for mask_male_label in [l + '_m' for l in mask_label_name_list]], 
           'fm': [d[mask_male_label] for mask_male_label in [l + '_fm' for l in mask_label_name_list]]}

bias = []

for i in range(5):
    bias.append(gen_arr['fm'][i] - gen_arr['m'][i])
    
gen_arr['bias(fm - m)'] = bias

import pandas as pd
df = pd.DataFrame(data=gen_arr, index=mask_label_name_list)

df

{'mask_worn_incorrectly_m': 453, 'surgical_mask_fm': 544, 'no_face_mask_fm': 496, 'cloth_mask_fm': 550, 'no_face_mask_m': 504, 'n95_mask_m': 461, 'n95_mask_fm': 539, 'surgical_mask_m': 456, 'mask_worn_incorrectly_fm': 547, 'cloth_mask_m': 450}


,m,fm,bias(fm - m)
cloth_mask,450,550,100
no_face_mask,504,496,-8
surgical_mask,456,544,88
n95_mask,461,539,78
mask_worn_incorrectly,453,547,94


In [3]:
from evaluation import downgrade_argmax_target_to_race, mask_label_name_list
from data_process.DatasetHelper import label_map_new_race
race_label_list = [downgrade_argmax_target_to_race(y) for y in data_y]

d = {}

for l in race_label_list:
    l_name = label_map_new_race[l]
    if d.get(l_name) == None:
        d[l_name] = 0
    d[l_name] += 1

print(d)

race_arr = {'caas': [d[mask_male_label] for mask_male_label in [l + '_caas' for l in mask_label_name_list]], 
           'afar': [d[mask_male_label] for mask_male_label in [l + '_afar' for l in mask_label_name_list]]}

bias = []

for i in range(5):
    bias.append(race_arr['afar'][i] - race_arr['caas'][i])
    
race_arr['bias(afar - caas)'] = bias

import pandas as pd
df = pd.DataFrame(data=race_arr, index=mask_label_name_list)

df

{'mask_worn_incorrectly_afar': 300, 'surgical_mask_caas': 600, 'no_face_mask_caas': 525, 'cloth_mask_caas': 600, 'no_face_mask_afar': 475, 'mask_worn_incorrectly_caas': 700, 'n95_mask_afar': 400, 'surgical_mask_afar': 400, 'cloth_mask_afar': 400, 'n95_mask_caas': 600}


,caas,afar,bias(afar - caas)
cloth_mask,600,400,-200
no_face_mask,525,475,-50
surgical_mask,600,400,-200
n95_mask,600,400,-200
mask_worn_incorrectly,700,300,-400
